<a href="https://colab.research.google.com/github/koted0/Project-Gozle/blob/main/Project_Gozle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Package Instalations, Imports and Const Links

In [ ]:
!pip install selenium
!pip install yt-dlp
!apt update
!apt install chromium-chromedriver -y
!apt install ffmpeg
!rm -R sample_data/
!clear

In [ ]:
import requests
import re
import os
import json

from requests.structures import CaseInsensitiveDict
# from yt_dlp import YoutubeDL
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from google.colab import userdata
from queue import Queue
from selenium.common import ElementNotInteractableException, NoSuchElementException

In [ ]:
GOZLE_URL = "https://disk.gozle.com.tm/login"

## Downloader

In [ ]:
class Downloader:
    def __init__(self):
        self.download_list: Queue = Queue()
        self.video_quality: dict = { "hd": "res:720", "fhd": "res:1080", 'uhd': "res:2160" }
        self.youtube_pattern = re.compile(r"(?:https?:\/\/)?(?:www\.)?youtu\.?be(?:\.com)?\/?.*(?:watch|embed)?(?:.*v=|v\/|\/)([\w\-_]+)\&?")


    def _get_headers(self, url: str) -> requests.Response:
        with requests.head(url) as response:
            try:
                return response.headers
            except Exception:
                print("\n\033[1;33m [WARNING]: No headers in response \n\033[0m")
                return None

    def _get_filename_from_header(self, header: CaseInsensitiveDict) -> str | None:
        try:
            content_disposition = header.get('content-disposition', '')
            if "filename=" in content_disposition:
                filename = content_disposition.split("filename=")[-1].strip('"')
            else:
                print("\n\033[1;33m [WARNING]: No filename in headers \033[0m")
                return None
        except Exception:
            print("\n\033[1;33m [WARNING]: Unexpected error occurred while extracting filename \n\033[0m")
            return None

    def _get_filename_from_url(self, url: str) -> str:
        filename = url.split("/")[-1]
        return filename if "?" not in filename else filename.split("?")[0]

    def _get_proper_name(self, url: str) -> str:
        headers = self._get_headers(url)
        if headers:
            filename = self._get_filename_from_header(headers)
            if filename:
                return filename
            return self._get_filename_from_url(url)

    def _is_file_exists(self, filename: str, path: str) -> bool:
        file_path = os.path.join(path, filename)
        return os.path.exists(file_path)

    def _download_other(self, url: str) -> None:
        path = "/content/others/"
        filename = self._get_proper_name(url)
        if self._is_file_exists(path, filename):
            print(f"\n\033[1;37m [INFO]: File: {filename} exists. Skipping \033[0m")
        else:
            !curl -o {path}{self._get_proper_name(url)} -L --create-dirs {url}

    # TODO: перехватывать название видео и вес
    def _download_youtube(self, url: str, quality_key: str) -> None:
        path = "/content/videos/"
        quality = self.video_quality[quality_key] if quality_key else "res:720"
        !yt-dlp -S {quality} {url} -P {path}

    def add_url(self, url: str, quality_key: str = None) -> None:
        """Add a URL to the download list."""
        self.download_list.put((url, quality_key))

    def add_downloads_from_file(self, path2file: str) -> None:
        with open(path2file, 'r') as file:
            [self.add_url(url.strip()) for url in file]

    def download(self) -> None:
        """Download all URLs in the download list."""
        while not self.download_list.empty():
            url, quality_key = self.download_list.get()
            if re.match(self.youtube_pattern, url):
                self._download_youtube(url, quality_key)
            else:
                self._download_other(url)
        else: print("\n\033[1;32m [INFO]: Downloading Finished \033[0m")


## Gozle Disk Uploader (Selenium)

In [ ]:
class Uploader:
    def __init__(self):
        self._gozle_username = userdata.get("username")
        self._gozle_password = userdata.get("password")
        self.archive_paths: dict = {}

    def _set_webdriver_options(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.set_capability("goog:LoggingPrefs", {"performance": "ALL"})
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(options=chrome_options)
        self.driver.maximize_window()
        self.wait = WebDriverWait(self.driver, 10)

    def login(self):
        self._set_webdriver_options()
        self.driver.get(GOZLE_URL)
        self.driver.find_element(By.NAME,value='email').send_keys(self._gozle_username)
        self.driver.find_element(By.NAME,value='password').send_keys(self._gozle_password)
        self.driver.find_element(By.CSS_SELECTOR, 'button[type = \'submit\']').click()

    # TODO: доделать realtime отслеживание количества загружаемых файлов и вес.
    # def _how_much_is_left(self):
    #     uploading_menu = driver.find_element(By.XPATH, '//*[@id="root"]/div[2]/div[5]/div[1]')
    #     for perf_entry in self.driver.get_log("performance"):
    #         perf_entry["message"] = json.loads(perf_entry["message"])["message"]
    #         print(perf_entry)

    def _archive_data(self):
        print(f"\n\033[1;33m [INFO]: Starting archiving \033[0m")
        if os.path.isdir("/content/others/"):
            !zip -r -m -s 4400m others /content/others/
        if os.path.isdir('/content/videos/'):
            !zip -r -m -s 4400m videos /content/videos/

        cwd = os.getcwd()
        files = [file for file in os.listdir(cwd) if not file.startswith('.') \
                 and not file.endswith('.png')]
        for file in files:
            self.archive_paths[file] = os.path.join(cwd, file)

    def upload(self):
        self._archive_data()
        max_attempts = 3

        # модифицировать код для использования встроенных возможностей selenium ждать и перехватывать ошибки.
        for filename, path in self.archive_paths.items():
            for attempt in range(max_attempts):
                try:
                    self.driver.find_element (By.XPATH, "//div//div[2]//div//div//div//button[starts-with(@id, ':r')]").click()
                    self.driver.find_element(By.XPATH, "//div[@data-value='uploadFiles']").click()
                    self.driver.find_element(By.CSS_SELECTOR, value='input[type = \'file\']').send_keys(path)
                    print(f"\n\033[1;37m [INFO]: Uploading {filename} \033[0m")
                    break
                except (NoSuchElementException, ElementNotInteractableException) as error:
                    print(f"\033[1;33m [WARNING]: Attempt {attempt+1} failed with error: {error}")
                    if attempt+1 == max_attempts:
                        print(f"\033[1;31m [ERROR]: Max attempts reached, moving to next file")
            # self._how_much_is_left()
            self.screenshot()

    def refresh_page(self):
        self.driver.refresh()

    def screenshot(self):
        self.driver.save_screenshot("uploading.png")

# Instances

In [ ]:
d = Downloader()
u = Uploader()

In [ ]:
d.add_url("https://downloads.sourceforge.net/project/xiaomi-eu-multilang-miui-roms/xiaomi.eu/HyperOS-WEEKLY-RELEASES/OS1.0.23.12.18.DEV/xiaomi.eu_multi_XM12_OS1.0.23.12.18.DEV_os1-14.zip?ts=gAAAAABliAI6v6rMCUqEdZKslTycGboExpfkX6HNGPCSgjQBI5JA5oEDIbECjMAA8oz72x1bwbuvxB_JnIG-Jk-5Jm2v6E8zyg%3D%3D&use_mirror=cfhcable&r=https%3A%2F%2Fsourceforge.net%2Fprojects%2Fxiaomi-eu-multilang-miui-roms%2Ffiles%2Fxiaomi.eu%2FHyperOS-WEEKLY-RELEASES%2FOS1.0.23.12.18.DEV%2Fxiaomi.eu_multi_XM12_OS1.0.23.12.18.DEV_os1-14.zip%2Fdownload")
# d.add_downloads_from_file("downloads.txt")
d.download()


 [INFO]: File: xiaomi.eu_multi_XM12_OS1.0.23.12.18.DEV_os1-14.zip exists. Skipping 

 [INFO]: Downloading Finished 


In [ ]:
u.login()

In [ ]:
u.upload()


 [INFO]: Starting archiving 
  adding: content/others/ (stored 0%)
  adding: content/others/1d6e24172336fb849ac63305918921f7e667d49a20f752c5081a6997926b686b.mp4 (deflated 0%)
  adding: content/others/a7ba6c7e66495c655b5d4b150a8fec2c351ac328c9df554dda549f32e062dc98.zip (stored 0%)
  adding: content/others/Shadowheart-Auxtasy-Baldurs-Gate-3-Animated-Hentai-3D-CGI-Video.mp4 (deflated 1%)
  adding: content/others/0bff4e1bb7e5ed6bba854006dca7ff98ba5a9e05443b67e096b19ea1b60bd719.mp4 (deflated 0%)
  adding: content/others/3ef438d124bcf687371fdbb73232f700bb0fedc641fe73f95d4f579d7e206169.mp4 (deflated 0%)
  adding: content/others/327c3915c886f956f83d88e558dea38a6c6d666b891e6c22d0ee399d1b8347b2.mp4 (deflated 0%)
  adding: content/others/06103a424d37ab4b72f75eb3c3797d6ef944dd5f1e695ecbdbd0289623a96d02.mp4 (deflated 0%)
  adding: content/others/d512dc508cbbde2a9f7872f42cdaa24f9a1933968035c346966ec243cb1ae037.mp4 (deflated 0%)

 [INFO]: Uploading others.zip 

 [INFO]: Uploading downloads.txt 


In [ ]:
u.screenshot()